In [1]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [3]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("training_data_all_offensive_offensive.csv")
print(data_initial.shape)

(89137, 633)


In [4]:
data_no_na = data_initial.dropna() # data should already have no NAs due to numerical imputation
data = data_no_na.select_dtypes(['number'])
x = data.loc[:, data.columns != 'net_xg']
x = x.loc[:, x.columns != 'net_xg']
y = data['net_xg']

X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.6, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, train_size = 0.5, test_size=0.5, random_state=45)

# Above two lines in unison accomplish a 60-20-20 split for train-validation-test

In [5]:
device_selection = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_selection)
print(device_selection)
print(device)

if torch.cuda.is_available():
        torch.cuda.empty_cache()

train_batch_size = 1000
val_batch_size = 1000
test_batch_size = 1000

cuda
cuda


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.df.iloc[idx, -1:].values, dtype=torch.float32)
        return x, y

train_df =  pd.concat([X_train, y_train], axis=1)  # pandas DataFrame containing training data
valid_df = pd.concat([X_val, y_val], axis=1) # pandas DataFrame containing validation data
test_df = pd.concat([X_test, y_test], axis=1)  # pandas DataFrame containing test data
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

# Credit to ChatGPT for generating this cell of code in response to the following prompt: design a dataloader that accepts the output of a pandas train-validation-test split. Basically write code for a PyTorch dataloader that accepts pandas dataframes as arguments

In [25]:
class VanillaNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(VanillaNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(self.input_size, 1000)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.batch_norm_1000 = nn.BatchNorm1d(1000)
        self.batch_norm_500 = nn.BatchNorm1d(500)
        self.batch_norm_400 = nn.BatchNorm1d(400)
        self.batch_norm_300 = nn.BatchNorm1d(300)
        self.batch_norm_200 = nn.BatchNorm1d(200)
        self.batch_norm_100 = nn.BatchNorm1d(100)

        self.fc2 = nn.Linear(1000, 1000)

        self.fc3 = nn.Linear(1000, 1000)

        self.fc4 = nn.Linear(1000, 500)

        self.fc5 = nn.Linear(500, 500)

        self.fc6 = nn.Linear(500, 400)

        self.fc7 = nn.Linear(400, 300)

        self.fc8 = nn.Linear(300, 200)

        self.fc9 = nn.Linear(200, 100)

        self.fc10 = nn.Linear(100, 1)

    def forward(self, x):
        #print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_500(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_500(x)

        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_400(x)

        x = self.fc7(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_300(x)

        x = self.fc8(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_200(x)

        x = self.fc9(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_100(x)

        x = self.fc10(x)
        
        return x


In [26]:
num_epochs = 20
trial_count = 30

learning_rate = 0.01 # in last optimization, best lr ended up being 0.0016
loss_func = nn.MSELoss()

def train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device):
    training_losses = np.array([])
    validation_losses = np.array([])
    for epoch in range(num_epochs):
        print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
        ########################### Training #####################################
        model.train(True)
        train_loss = 0
        count = 0
        for batch, (X, y) in enumerate(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            optimizer.zero_grad()
            prediction = model(X).to(device)
            loss = loss_func(prediction, y.view(-1, 1))
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.float().item()
            count += train_batch_size
        training_losses = np.append(training_losses, train_loss / count)
        print("Training loss:", train_loss / count)
        #model.train(False)

        ########################### Validation #####################################
        model.eval()
        val_loss = 0
        count = 0
        with torch.no_grad():
            for (X, y) in valid_loader:
                X = X.to(device)
                y = y.to(device)
                prediction = model(X).to(device)
                loss = loss_func(prediction, y.view(-1, 1))
                val_loss += loss.float().item()
                count += val_batch_size
        validation_losses = np.append(validation_losses, val_loss / count)
        print("Validation loss:", val_loss / count)
    return validation_losses[-1]

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hidden_size = trial.suggest_int("hidden_size", 300, 600)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    print("learning rate:", lr)
    print("weight_decay:", weight_decay)
    print("hidden_size:", hidden_size)
    print("dropout_rate:", dropout_rate)

    model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

input_dim = 630
output_dim = 1
hidden_dim = 300
dropout = 0.1
model = VanillaNeuralNet(input_dim, hidden_dim, dropout)

In [28]:
hidden_size = 500
dropout_rate = 0.25
weight_decay = 0.000001
learning_rate = 0.01

# Above inspired by last run best params, which were:
# {'lr': 0.0016359428701992723, 'weight_decay': 3.4162832861696797e-06, 'hidden_size': 493, 'dropout_rate': 0.4524812840729844}
num_epochs = 20
model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()


# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_off_off_state_dict2.pt")
# save the trained model
torch.save(model, "faceoffs_off_off2.pt")

Epoch 1 Training loss: 0.06633286259171588
Epoch 2 Training loss: 0.014052047273489061
Epoch 3 Training loss: 0.014114446644843728
Epoch 4 Training loss: 0.014063433330092166
Epoch 5 Training loss: 0.014055595440031201
Epoch 6 Training loss: 0.01423367078381556
Epoch 7 Training loss: 0.014168422203511
Epoch 8 Training loss: 0.014146804257675453
Epoch 9 Training loss: 0.014276746384523533
Epoch 10 Training loss: 0.014209460518840287
Epoch 11 Training loss: 0.013981369410262064
Epoch 12 Training loss: 0.013870957031570099
Epoch 13 Training loss: 0.013855739582881884
Epoch 14 Training loss: 0.013896045516486521
Epoch 15 Training loss: 0.013891189445362048
Epoch 16 Training loss: 0.013857890196420529
Epoch 17 Training loss: 0.013854449166467896
Epoch 18 Training loss: 0.013847928614942011
Epoch 19 Training loss: 0.01391471170440868
Epoch 20 Training loss: 0.013873228734290158


In [ ]:
# Optuna Hyperparameter Tuning
import optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=trial_count)
print(study.best_params)

In [29]:
best_params = study.best_trial.params
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

num_epochs = 100 # training for more epochs on best model (which will be saved)

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params2.pt")
# save the trained model
torch.save(model, "faceoffs_best_model2.pt")

NameError: name 'study' is not defined

In [30]:
test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print("Loss:", test_loss)
print(f"R-squared Coefficient: {r2:.4f}")


Test Loss: 0.0137
True labels: [0.01828337, 0.0, 0.0, 0.0, -0.0148857, 0.0, 0.0196506, 0.0, 0.3549229, 0.0, -0.0576078, -0.32548, 0.0, 0.0, -0.00856119, 0.182302, -0.0089685, -0.0100522, 0.2010707, -0.14259318, 0.0, 0.00871652, 0.0, 0.17065948, 0.0243813, -0.0125711, 0.0308997, 0.016226014, 0.0315559, 0.0, 0.0, 0.036871, 0.2126759, 0.0, 0.0, 0.0, -0.0616374, 0.0099204, 0.0, 0.0, 0.0, -0.1184434, 0.0, 0.2269262, 0.0, 0.0, 0.0021875, 0.01323375, 0.0134349, -0.0859904, -0.15662608, 0.0, -0.262752, 0.0, -0.0582621, 0.0, 0.0, -0.423586, -0.01756836, 0.0710411, -0.0334358, -0.036931, 0.0, 0.0, 0.0742537, 0.0, 0.1574826, 0.147533, 0.00858838, 0.0292749, 0.0, 0.0, 0.02323446, 0.0, 0.023919, 0.16008036, -0.00943772, -0.1786999, -0.00869689, -0.01886305, 0.0, 0.1488199, 0.0404832, 0.3313081, 0.0217645, 0.0, 0.0, 0.0, 0.0, -0.087195, -0.20890877, 0.0449345, 0.0, 0.0, 0.0, 0.1112595, 0.0, -0.1014313, 0.00861839, 0.0574316, 0.1536152, 0.0, -0.0042232, 0.0, 0.0708015, 0.08621225, 0.0, 0.0330144, -0.

In [23]:
num_epochs = 20

learning_rate = 0.01
loss_func = nn.MSELoss()
input_dim = 950
best_params = {'lr': 0.008527084096275877, 'weight_decay': 6.071847158024609e-06, 'hidden_size': 352, 'dropout_rate': 0.4894615406787279}
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params_revisited.pt")
# save the trained model
torch.save(model, "faceoffs_best_model_revisited.pt")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x950 and 799x352)

In [ ]:
test_loss = 0
y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print(f"R-squared Coefficient: {r2:.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x799 and 529x352)